In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

### upload year 2010 daily data

In [3]:
ur_files = ddf.read_csv('DailyTemp/DailyTemp_2008.csv', dtype = {'fips': str, 'STATION': 'object'})
temp = ur_files.compute()
temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
temp['year'] = '2008'
temp.head()

,STATION,LATITUDE,LONGITUDE,NAME,YEAR,DATE,TEMP,MIN,MAX,year
0,826099999,0.0,0.0,WXPOD8270,2008,2008-01-01,36.5,19.4,55.9,2008
1,826099999,0.0,0.0,WXPOD8270,2008,2008-01-02,37.0,24.8,54.0,2008
2,826099999,0.0,0.0,WXPOD8270,2008,2008-01-03,11.9,6.8,25.0,2008
3,826099999,0.0,0.0,WXPOD8270,2008,2008-01-04,31.8,8.6,59.0,2008
4,826099999,0.0,0.0,WXPOD8270,2008,2008-01-05,48.4,33.8,57.2,2008


In [4]:
temp['YEAR'].unique()

array([2008])

In [5]:
def rangeBelow30(temp):
    if temp < 30:
        return 1
    else:
        return 0

In [6]:
def range30To40(temp):
    if temp < 40 and temp >= 30:
        return 1
    else:
        return 0

In [7]:
def range40To50(temp):
    if temp < 50 and temp >= 40:
        return 1
    else:
        return 0

In [8]:
def range50To60(temp):
    if temp < 60 and temp >= 50:
        return 1
    else:
        return 0

In [9]:
def range60To70(temp):
    if temp < 70 and temp >= 60:
        return 1
    else:
        return 0

In [10]:
def range70To80(temp):
    if temp < 80 and temp >= 70:
        return 1
    else:
        return 0

In [11]:
def range80To90(temp):
    if temp < 90 and temp >= 80:
        return 1
    else:
        return 0

In [12]:
def rangeOver90(temp):
    if temp >= 90:
        return 1
    else:
        return 0

In [13]:
def changeToNone(value):
    if value == 999.9 or value == 9999.9 or value == 99.99 or value == 99.9:
        return np.nan
    else:
        return value

In [14]:
temp['TEMP'] = temp['TEMP'].apply(changeToNone)

In [17]:
temp['<30'] = temp['TEMP'].apply(rangeBelow30)
temp['30-40'] = temp['TEMP'].apply(range30To40)
temp['40-50'] = temp['TEMP'].apply(range40To50)
temp['50-60'] = temp['TEMP'].apply(range50To60)
temp['60-70'] = temp['TEMP'].apply(range60To70)
temp['70-80'] = temp['TEMP'].apply(range70To80)
temp['80-90'] = temp['TEMP'].apply(range80To90)
temp['>=90'] = temp['TEMP'].apply(rangeOver90)

In [18]:
temp.head()

,STATION,LATITUDE,LONGITUDE,NAME,YEAR,DATE,TEMP,MIN,MAX,year,<30,30-40,40-50,50-60,60-70,70-80,80-90,>=90
0,826099999,0.0,0.0,WXPOD8270,2008,2008-01-01,36.5,19.4,55.9,2008,0,1,0,0,0,0,0,0
1,826099999,0.0,0.0,WXPOD8270,2008,2008-01-02,37.0,24.8,54.0,2008,0,1,0,0,0,0,0,0
2,826099999,0.0,0.0,WXPOD8270,2008,2008-01-03,11.9,6.8,25.0,2008,1,0,0,0,0,0,0,0
3,826099999,0.0,0.0,WXPOD8270,2008,2008-01-04,31.8,8.6,59.0,2008,0,1,0,0,0,0,0,0
4,826099999,0.0,0.0,WXPOD8270,2008,2008-01-05,48.4,33.8,57.2,2008,0,0,1,0,0,0,0,0


In [19]:
temp = temp.groupby(['STATION', 'LATITUDE', 'LONGITUDE']).sum()
temp = temp.reset_index()
temp.head()

,STATION,LATITUDE,LONGITUDE,YEAR,TEMP,MIN,MAX,<30,30-40,40-50,50-60,60-70,70-80,80-90,>=90
0,10000199999,51.602414,6.142172,732920,19036.8,16307.8,21338.8,5,53,114,75,98,20,0,0
1,1001099999,70.933333,-8.666667,734928,12176.4,10974.3,13258.0,117,158,91,0,0,0,0,0
2,1001499999,59.791925,5.340850,445776,11226.1,10256.6,11928.8,0,42,65,68,37,10,0,0
3,10018099999,54.913250,8.340472,730912,18962.6,17058.8,19026.2,4,48,119,86,88,18,1,0
4,1003099999,77.000000,15.500000,734928,9600.6,8117.6,10952.1,206,129,31,0,0,0,0,0


In [20]:
temp = temp[['STATION', 'LATITUDE', 'LONGITUDE', '<30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '>=90']]
temp['year'] = '2008'
temp = temp.rename(columns = {'STATION': 'station', 'LATITUDE': 'latitude', 'LONGITUDE': 'longitude'})
temp.head()

,station,latitude,longitude,<30,30-40,40-50,50-60,60-70,70-80,80-90,>=90,year
0,10000199999,51.602414,6.142172,5,53,114,75,98,20,0,0,2008
1,1001099999,70.933333,-8.666667,117,158,91,0,0,0,0,0,2008
2,1001499999,59.791925,5.340850,0,42,65,68,37,10,0,0,2008
3,10018099999,54.913250,8.340472,4,48,119,86,88,18,1,0,2008
4,1003099999,77.000000,15.500000,206,129,31,0,0,0,0,0,2008


In [21]:
temp.to_csv('TempCount/Temp_Count_2008.csv')

In [15]:
years = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

In [16]:
for year in years:
    ur_files = ddf.read_csv('DailyTemp/DailyTemp_' + year + '.csv', dtype = {'STATION': 'object'})
    temp = ur_files.compute()
    temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
    temp['TEMP'] = temp['TEMP'].apply(changeToNone)
    temp['MIN'] = temp['MIN'].apply(changeToNone)
    temp['MAX'] = temp['MAX'].apply(changeToNone)
    
    temp['mean_temp_below_30'] = temp['TEMP'].apply(rangeBelow30)
    temp['mean_temp_30-40'] = temp['TEMP'].apply(range30To40)
    temp['mean_temp_40-50'] = temp['TEMP'].apply(range40To50)
    temp['mean_temp_50-60'] = temp['TEMP'].apply(range50To60)
    temp['mean_temp_60-70'] = temp['TEMP'].apply(range60To70)
    temp['mean_temp_70-80'] = temp['TEMP'].apply(range70To80)
    temp['mean_temp_80-90'] = temp['TEMP'].apply(range80To90)
    temp['mean_temp_over_90'] = temp['TEMP'].apply(rangeOver90)
    
    temp['min_temp_below_30'] = temp['MIN'].apply(rangeBelow30)
    temp['min_temp_30-40'] = temp['MIN'].apply(range30To40)
    temp['min_temp_40-50'] = temp['MIN'].apply(range40To50)
    temp['min_temp_50-60'] = temp['MIN'].apply(range50To60)
    temp['min_temp_60-70'] = temp['MIN'].apply(range60To70)
    temp['min_temp_70-80'] = temp['MIN'].apply(range70To80)
    temp['min_temp_80-90'] = temp['MIN'].apply(range80To90)
    temp['min_temp_over_90'] = temp['MIN'].apply(rangeOver90)
    
    temp['max_temp_below_30'] = temp['MAX'].apply(rangeBelow30)
    temp['max_temp_30-40'] = temp['MAX'].apply(range30To40)
    temp['max_temp_40-50'] = temp['MAX'].apply(range40To50)
    temp['max_temp_50-60'] = temp['MAX'].apply(range50To60)
    temp['max_temp_60-70'] = temp['MAX'].apply(range60To70)
    temp['max_temp_70-80'] = temp['MAX'].apply(range70To80)
    temp['max_temp_80-90'] = temp['MAX'].apply(range80To90)
    temp['max_temp_over_90'] = temp['MAX'].apply(rangeOver90)
    
    temp = temp.groupby(['STATION', 'LATITUDE', 'LONGITUDE']).sum()
    temp = temp.reset_index()
    temp = temp[['STATION', 'LATITUDE', 'LONGITUDE', 'mean_temp_below_30', 
                 'mean_temp_30-40', 'mean_temp_40-50', 'mean_temp_50-60', 
                 'mean_temp_60-70', 'mean_temp_70-80', 'mean_temp_80-90', 
                 'mean_temp_over_90', 'min_temp_below_30', 'min_temp_30-40', 
                 'min_temp_40-50', 'min_temp_50-60', 'min_temp_60-70', 
                 'min_temp_70-80', 'min_temp_80-90', 'min_temp_over_90',
                 'max_temp_below_30', 'max_temp_30-40', 'max_temp_40-50', 
                 'max_temp_50-60', 'max_temp_60-70', 'max_temp_70-80',
                 'max_temp_80-90', 'max_temp_over_90']]
    temp['year'] = year
    temp = temp.rename(columns = {'STATION': 'station', 'LATITUDE': 'latitude', 'LONGITUDE': 'longitude'})
    temp.to_csv('Temp_Count_' + year + '.csv')